In [1]:
import numpy as np
import pandas as pd
import csv
csv.field_size_limit(6000000)

131072

## 讀取txt資料

In [2]:
df = pd.read_csv('raw_data/VW_NTNU.txt', sep="\t", encoding='utf-8', engine='c', usecols=['F4', 'F9', 'F32'],
                 dtype={'F4': str, 'F9': str, 'F32': str}).rename(columns={'F4': 'source', 'F9': 'summary', 'F32': 'document'})

print("Shape:", df.shape)
df.sample(5)

Shape: (5957120, 3)


,source,summary,document
988029,mobile01,汐止合康雲極討論,大大很早就買了嗎？半年前去看時，代銷說兩房幾乎都賣完了⋯⋯\n\n
3541048,中時電子報_即時新聞,《新金融觀察》IPO審核上週三演「孤獨一枝」,團滅——這是對上周來到發審委上會公司最恰當的形容。上周共有18家擬上市公司上會，結果是上週二...
2229502,facebook粉絲團,【微型住宅：革命性創新設計大潮】由於空間昂貴，住房稀缺，出於,【微型住宅：革命性創新設計大潮】由於空間昂貴，住房稀缺，出於經濟考慮，一些建築師在設計住宅時...
4192380,Ptt,Re: [閒聊] 京都1度，北越8度，台灣10度就喊冷？,"<div class=""bbs-screen bbs-content"" id=""main-c..."
3060783,facebook粉絲團,Jasmine Lim,Jasmine Lim


## 分析資料來源與比例

In [3]:
count = df.groupby(['source']).size().sort_values(ascending=False)
pd.concat([count, count.map(lambda x: '%.4f' % (x/df.shape[0]*100) + '%')], axis=1, keys=['資料筆數', '比例'])

,資料筆數,比例
source,,
facebook粉絲團,3888087,65.2679%
mobile01,501221,8.4138%
Ptt,400419,6.7217%
Youtube關鍵字搜尋,157038,2.6361%
Dcard,135192,2.2694%
Google Play App 評價,116089,1.9487%
伊莉討論區,97673,1.6396%
TWitter搜尋,76369,1.2820%
yahoo新聞,60471,1.0151%


## 選取文本自20個新聞網
### 每個來源抽樣2000筆

In [4]:
SELECTIONS = ['yahoo新聞', 'pchome新聞', '聯合新聞網', '蘋果日報_即時新聞', '東森新聞',
              '自由時報即時', '中時電子報_即時新聞', '鉅亨網', 'TVBS新聞', '三立新聞',
              '中央社','MoneyDJ理財網', '蘋果日報', '聯合新聞網_即時新聞', '華視新聞',
              '民視新聞', '中時電子報', '工商時報', '中廣新聞網', '台視新聞']
SAMPLE_NUM = 2000

In [5]:
select_df = pd.DataFrame()
for source in SELECTIONS:
    sample_df = df.loc[df['source'] == source].sample(SAMPLE_NUM)
    select_df = pd.concat((select_df, sample_df))
    
select_df = select_df.reset_index(drop=True)
select_df['id'] = select_df.index
select_df = select_df[['id', 'source', 'summary', 'document']]

print("Shape:", select_df.shape)

Shape: (40000, 4)


In [6]:
select_df.to_pickle("data/corpus.pkl")

### 提取來源中全部的文本

In [7]:
select_all_df = df.loc[df.source.isin(SELECTIONS)].reset_index(drop=True)
select_all_df['id'] = select_all_df.index
select_all_df = select_all_df[['id', 'source', 'summary', 'document']]

print("Shape:", select_all_df.shape)

Shape: (367027, 4)


In [8]:
select_all_df.to_pickle('word2vec/news-corpus.pkl')